In [ ]:
from datetime import datetime, timedelta
import json
import markdown
import pandas
from pathlib import Path
import re

In [ ]:
# Specify input RSS file from https://www.meetup.com/hacksburgva/events/rss/ (Must be signed into meetup)
file = 'rss.txt'

# Change extension to json for output
out_file = Path(file).with_suffix('.json')

# Read data from input file, drop weekly meeting events
data = pandas.read_xml(file, xpath=".//item")
data.drop(data[data["title"] == "Weekly Meeting"].index, inplace=True)

In [ ]:
# Define regex for parsing description field (includes date, time, description, number of attendees, price, etc)
expression = re.compile('.*<p><p>(?P<description>.*)<\/p> <\/p> <p>(?P<location>.*)<\/p> <p>(?P<date>.*)<\/p> <p>(?P<attendees>.*)<\/p> <p>Price: (?P<price>[0-9]+).*<\/p> <p>(?P<url>.*)<\/p>.*')

# Initialize empty frame for collecting events
output = pandas.DataFrame()	

# Iterate through events
# # Notes:
# - No subtitles are generated
# - Meetup does not include year in timestamp, Dates may be innaccurate near end of year
# - End time assumed to be 3 hours after start
# - By default assuming onsite location, not offered online, offered in person
# - Member price $5 less than meetup price
# - No image attached

for index, row in data.iterrows():
    description_groups = expression.match(row["description"]).groupdict()
    date_time = datetime.strptime(description_groups['date'], '%A, %B %d at %I:%M %p').replace(year=datetime.today().year)
    temp = {"title": row['title'],
			"subtitle": "",
			"description": markdown.markdown(description_groups['description']),
			"date": date_time.strftime('%Y-%m-%d'),
			"start_time": date_time.strftime('%-I:%M%p').lower(),
			"end_time": (date_time + timedelta(hours=3)).strftime('%-I:%M%p').lower(),
			"offsite_location": None,
			"offered_online": False,
			"offered_in_person": True,
			"member_price": int(description_groups['price'])-5,
			"non_member_price": int(description_groups['price']),
			"image": "",
			"meetup_link": row['guid']}
    output = output.append(temp, ignore_index=True)

In [ ]:
# Write to output file
# Escapes forward slashed by default, has no impact on generared html
output.to_json(orient="records",path_or_buf=out_file)